# Web Scraping www.drogueriascolsubsidio.com

In [1]:
import os
os.chdir(r'C:\Users\Fsalinas\Documents\GitHub\boticarios')

In [196]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import urllib.request
from contextlib import closing
from datetime import datetime as dt
import pandas as pd
import time
import json

_________
### Web Crawler Categorias
Extraé urls de categorias que contienen las url de los productos para posteriormente extraer información de producto.

In [10]:
# Define rutas a urls y archivos
chromedriver = './web_scraping/chromedriver/chromedriver.exe'
url_principal = 'https://www.drogueriascolsubsidio.com'

# Define si el navegador estará visible durante el proceso
hide_browser = False

# Aplica opciones al navegador para evitar cargar recursos innecesarios
options = Options()
options.add_argument('--ignore-certificate-errors')
if hide_browser: options.add_argument('--headless')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option('prefs',{'profile.managed_default_content_setings.images':2})

# Crea el objeto del navegador con el que se realizará la interacción
with closing(Chrome(executable_path = chromedriver, options=options)) as navegador:

    # Navega a la URL principal donde se extraerán las URLs de categorias y subcategorias de producto
    navegador.get(url_principal)
    soup = BeautifulSoup(navegador.page_source, 'html.parser')

    # Obtiene una lista de las etiquetas "ul" asociadas a las categorias de producto
    # Fuente: investigación en las particularidades de la construcción del sitio web, es suceptible a fallos en caso de que
    # la estructura (etiquetas) del sitio web cambie.
    lista_categorias = soup.find_all('ul',{'class':'categoria-container'})
    dc_cat_url = {cat.find_all('a')[1].text.lower().replace('ver ', ''):cat for x, cat in enumerate(lista_categorias)}

    #Extrae la lista de URLs de las etiquetas "ul"
    lista_urls = []
    for cat in dc_cat_url.keys():
        lista_urls += [x.get_attribute_list('href')[0] for x in dc_cat_url[cat].find_all('a')]

    lista_urls = list(dict.fromkeys(lista_urls))

# Guarda la lista de URLs en un archivo csv
with open('./web_scraping/data/cat_urls.csv', 'w+') as f:
    f.write('\n'.join(lista_urls))

_________
### Web Crawler Productos
Extraé urls de productos a partir de las URLs de categorias del archvio csv.

In [3]:
# Define rutas a urls y archivos
chromedriver = './web_scraping/chromedriver/chromedriver.exe'
url_principal = 'https://www.drogueriascolsubsidio.com'

# Define si el navegador estará visible durante el proceso
hide_browser = False

# Aplica opciones al navegador para evitar cargar recursos innecesarios
options = Options()
options.add_argument('--ignore-certificate-errors')
if hide_browser: options.add_argument('--headless')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option('prefs',{'profile.managed_default_content_setings.images':2})

In [4]:
# Leer el archivo con las url de categorias
with open('./web_scraping/data/cat_urls.csv', 'r') as f:
    cat_urls = f.read()
cat_urls = cat_urls.split('\n')

In [5]:
# Función para obtener todas las url de producto ubicadas en una url
def obtiene_url_productos(navegador):
    soup = BeautifulSoup(navegador.page_source, 'html.parser')
    urls_prod = [x.get_attribute_list('href')[0] for x in soup.find_all('a') if x.get_attribute_list('href')[0]!=None and x.get_attribute_list('href')[0][-2:].lower()=='/p']
    urls_prod = [x.replace(url_principal, '') for x in urls_prod]
    return list(dict.fromkeys(urls_prod))

In [6]:
def obtiene_urls_producto(navegador, url, tiempo_espera_scroll=2):
    # Navega a cada URL de categoria o subcategoria donde se extraerán las URLs de producto
    navegador.get(url)

    # Obtener el tamaño de la pagina cargada
    ultima_altura = navegador.execute_script("return document.body.scrollHeight")

    lista_urls_producto, scroll_nro = [], 1
    while True:
        print(f'{dt.now().strftime("%H:%M:%S")} Scroll nro: {scroll_nro:,.0f}', end = '\r')
        # Scroll hasta el final de la pagina
        navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # captura las urls de la pagina hasta donde está cargada
        lista_urls_producto += obtiene_url_productos(navegador)

        # esperar a que la pagina cargue
        time.sleep(tiempo_espera_scroll)

        # Calculate new scroll height and compare with last scroll height
        nueva_altura = navegador.execute_script("return document.body.scrollHeight")
        if nueva_altura == ultima_altura:
            break
        ultima_altura = nueva_altura
        scroll_nro += 1

    lista_urls_producto = list(dict.fromkeys(lista_urls_producto))
    return lista_urls_producto

In [9]:
# Crea el objeto del navegador con el que se realizará la interacción
# Duración aproximada: 30 minutos
with closing(Chrome(executable_path = chromedriver, options=options)) as navegador:
    lista_urls_productos = []
    for cat in cat_urls:
        print(f'{dt.now().strftime("%H:%M:%S")} Procesando: {cat}')
        lp = obtiene_urls_producto(navegador, url_principal + cat, 2)
        print(f'{dt.now().strftime("%H:%M:%S")} Numero de urls de producto conseguidas:{len(lp):,.0f}')
        lista_urls_productos += lp

lista_urls_productos = list(dict.fromkeys(lista_urls_productos))
print(f'{dt.now().strftime("%H:%M:%S")} Numero de urls de producto totales conseguidas:{len(lista_urls_productos):,.0f}')
print('Primeras 10 urls de la lista completa:')
lista_urls_productos[:10]

23:31:02 Procesando: /medicamentos
23:31:12 Numero de urls de producto conseguidas:33
23:31:12 Procesando: /medicamentos/venta-libre
23:32:42 Numero de urls de producto conseguidas:783
23:32:42 Procesando: /medicamentos/venta-libre/alivio-del-dolor
23:33:04 Numero de urls de producto conseguidas:164
23:33:04 Procesando: /medicamentos/venta-libre/alivio-del-dolor/dolor-fuerte
23:33:18 Numero de urls de producto conseguidas:37
23:33:18 Procesando: /medicamentos/venta-libre/alivio-del-dolor/dolor-general
23:33:27 Numero de urls de producto conseguidas:31
23:33:27 Procesando: /medicamentos/venta-libre/alivio-del-dolor/abdominal-y-colico
23:33:32 Numero de urls de producto conseguidas:19
23:33:32 Procesando: /medicamentos/venta-libre/alivio-del-dolor/dolor-ninos
23:33:40 Numero de urls de producto conseguidas:26
23:33:40 Procesando: /medicamentos/venta-libre/botiquin-y-primeros-auxilios
23:34:06 Numero de urls de producto conseguidas:255
23:34:06 Procesando: /medicamentos/venta-libre/botiqu

NameError: name 'printlista_urls_productos' is not defined

In [10]:
print(f'{dt.now().strftime("%H:%M:%S")} Numero de urls de producto totales conseguidas:{len(lista_urls_productos):,.0f}')
print('Primeras 10 urls de la lista completa:')
lista_urls_productos[:10]

10:40:27 Numero de urls de producto totales conseguidas:7,510
Primeras 10 urls de la lista completa:


['/advil-max-400-mg-capsula-blanda-7702132008611/p',
 '/systane-complete-gotas-lubricantes-300651481228/p',
 '/benzirin-verde-aseptic-solucion-bucal-7702057012724/p',
 '/7702870002964gastrum-10-mg-tabletamedicamentosbienestar-digestivolafrancolfamotidina-7702870002964/p',
 '/proteina-barbarus-healthy-sports-19962798764/p',
 '/engystol-tableta-7707336720598/p',
 '/optive-fusion-solucion-oftalmica-7707236676087/p',
 '/7702132004460pfizerbienestar-digestivomedicamentosmareol-50-mg-tabletadimenhidrinato-7702132004460/p',
 '/7702132004644oferta-chapstick-medicado-duo-pack-barra-precio-especialmedicamentoscuidado-facialna-7702132004644/p',
 '/dolex-forte-nf-500-mg65-mg-tableta-recubierta-7703363005462/p']

In [12]:
# Guarda la lista de URLs en un archivo csv
with open('./web_scraping/data/prod_urls.csv', 'w+') as f:
    f.write('\n'.join(lista_urls_productos))

_________
### Web Scraping Productos
Extraé data de cada url de producto a partir de las URLs de producto del archvio csv.

In [270]:
# Define rutas a urls y archivos
chromedriver = './web_scraping/chromedriver/chromedriver.exe'
url_principal = 'https://www.drogueriascolsubsidio.com'

# Define si el navegador estará visible durante el proceso
hide_browser = False

# Aplica opciones al navegador para evitar cargar recursos innecesarios
options = Options()
options.add_argument('--ignore-certificate-errors')
if hide_browser: options.add_argument('--headless')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option('prefs',{'profile.managed_default_content_setings.images':2})

In [271]:
# Leer el archivo con las url de categorias
with open('./web_scraping/data/prod_urls.csv', 'r') as f:
    prod_urls = f.read()
prod_urls = prod_urls.split('\n')

In [272]:
def descarga_imagen(navegador, nombre):
    url_imagen_producto = BeautifulSoup(navegador.page_source, 'html.parser').find_all('img', {'class': 'class-img-product'})[0].get_attribute_list('src')[0]
    nombre_imagen = f'./web_scraping/data/img/{nombre}.jpg'
    urllib.request.urlretrieve(url_imagen_producto, nombre_imagen)
    return nombre_imagen

In [273]:
def obtiene_data_producto(navegador):
    soup_producto = BeautifulSoup(navegador.page_source, 'html.parser').find_all('div', {'class': 'container-data'})[0]
    
    fecha_hora_scraping = dt.now().strftime('%Y-%m-%d %H:%M:%S')
    
    try:
        titulo = soup_producto.find_all('h2')[0].text
        precio = {str(x).split('class="')[1].split('"')[0]:x.text for x in soup_producto.find_all('div', {'class': 'precio'})[0].find_all('div')}
    except:
        return {
            'url_producto': 'no encontrado',
            'fecha_hora_scraping': fecha_hora_scraping,
            'breadcumb': 'no encontrado',
            'titulo': 'no encontrado',
            'nombre_imagen': 'no encontrado',
            'presentacion': 'no encontrado',
            'precio': 'no encontrado',
            'descripcion': 'no encontrado',
            'atributos': 'no encontrado'
        }
    
    try:
        breadcumb = '|'.join([x.text.lower() for x in BeautifulSoup(navegador.page_source, 'html.parser').find_all('div', {'class': 'bread-crumb'})[1].find_all('li')])
        presentacion = soup_producto.find_all('div', {'class': 'ContentPresentaticones'})[0].text
        descripcion = soup_producto.find_all('div', {'class': 'productDescription'})[0].text
        atributos = [tuple(x.text.split(':')) for x in soup_producto.find_all('div', {'class': 'divProduct-especificaciones'})[0].find_all('div', {'class': 'especificaciones_item'})]
    except:
        return {
            'url_producto': 'no encontrado',
            'fecha_hora_scraping': fecha_hora_scraping,
            'breadcumb': 'no encontrado',
            'titulo': titulo,
            'nombre_imagen': titulo + '.jpg',
            'presentacion': 'no encontrado',
            'precio': precio,
            'descripcion': 'no encontrado',
            'atributos': 'no encontrado'
        }
    
    try:
        nombre_imagen = descarga_imagen(navegador, ''.join([x for x in titulo.replace(' ', '_') if x.isalnum() or x=='_']))
    except:
        return {
            'url_producto': url_producto,
            'fecha_hora_scraping': fecha_hora_scraping,
            'breadcumb': breadcumb,
            'titulo': titulo,
            'nombre_imagen': titulo + '.jpg',
            'presentacion': presentacion,
            'precio': precio,
            'descripcion': descripcion,
            'atributos': atributos
        }
    
    return {
        'url_producto': url_producto,
        'fecha_hora_scraping': fecha_hora_scraping,
        'breadcumb': breadcumb,
        'titulo': titulo,
        'nombre_imagen': nombre_imagen,
        'presentacion': presentacion,
        'precio': precio,
        'descripcion': descripcion,
        'atributos': atributos
    }

In [279]:
# Crea el objeto del navegador con el que se realizará la interacción
# navegador = Chrome(executable_path = chromedriver, options=options)
with closing(Chrome(executable_path = chromedriver, options=options)) as navegador:
    fecha = dt.now().strftime('%Y%m%d')
    if not os.path.exists(f'./web_scraping/data/json/{fecha}'): os.makedirs(f'./web_scraping/data/json/{fecha}')
    json_path = f'./web_scraping/data/json/{fecha}'

    for prod_url in prod_urls:
        print(f'{dt.now().strftime("%H:%M:%S")} Procesando: {prod_url}')
        url_producto = url_principal + prod_url
        navegador.get(url_producto)
        time.sleep(2)
        dc_prod = obtiene_data_producto(navegador)

        json_data = json.dumps(dc_prod, indent=4, sort_keys=False)
        with open(f'{json_path}/{dc_prod["nombre_imagen"].split("/")[-1].replace(".jpg",".json")}', mode='w+', encoding='latin-1') as f:
            f.write(json_data)

18:02:04 Procesando: /advil-max-400-mg-capsula-blanda-7702132008611/p
18:02:12 Procesando: /systane-complete-gotas-lubricantes-300651481228/p
18:02:17 Procesando: /benzirin-verde-aseptic-solucion-bucal-7702057012724/p
18:02:26 Procesando: /7702870002964gastrum-10-mg-tabletamedicamentosbienestar-digestivolafrancolfamotidina-7702870002964/p
18:02:31 Procesando: /proteina-barbarus-healthy-sports-19962798764/p
18:02:36 Procesando: /engystol-tableta-7707336720598/p
18:02:41 Procesando: /optive-fusion-solucion-oftalmica-7707236676087/p
18:02:45 Procesando: /7702132004460pfizerbienestar-digestivomedicamentosmareol-50-mg-tabletadimenhidrinato-7702132004460/p
18:02:50 Procesando: /7702132004644oferta-chapstick-medicado-duo-pack-barra-precio-especialmedicamentoscuidado-facialna-7702132004644/p
18:02:55 Procesando: /dolex-forte-nf-500-mg65-mg-tableta-recubierta-7703363005462/p
18:02:59 Procesando: /7702057100384minoxidil-forte-5--solucion-topica-tecnoquimicasmedicamentoscuidado-capilartecnoquimic

WebDriverException: Message: chrome not reachable
  (Session info: chrome=91.0.4472.101)
